# FEniCSx Failure Cases

In this tutorial, we will deviate slightly from our analytical tutorial and H_P refinement tutorial. We redid the problem as a nonlinear problem and this time we will be testing out fail cases where we either do not get the correct solution, or entirely error out. We will continue with our 3D example as well. While only 3 examples were provided, there are many more edge cases that need to be watched for.  Before getting into the problem, please ensure that FEniCSx and DolfinX are installed on your computer. For assistance with installation, please refer to the `readme.md` for tips on installing the platform, creating a conda environment, activating it (also in your terminal/visual studio session), and running the Jupyter Notebook.

## Failure Condition 1 - Incorrect Mesh Size

Lets take a look at our first example where Fenicsx fails. When we set our mesh size too small or too large, Fenicsx provides an incorrect answer, or will time out due to memory issues. A small mesh size (as we will see in this example below), will simply provide the incorrect solution to our bridge force problem. If you would like to see the flip side of this, set the number of elements to [1000, 1000, 1000] and the program will fail due to memory overloading.

In [ ]:
from dolfinx import mesh, fem, log, plot, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from mpi4py import MPI
import numpy as np
import ufl
import pyvista

# Bridge dimensions
L, W, H = 10.0, 1.0, 0.5  # Length, width, height
num_elements = [2, 1, 1]  # Coarser mesh resolution for P-mesh

# Create a 3D bridge mesh
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, W, H]], num_elements, mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))

# Material properties for bridge
rho = 7850  # Density
E = default_scalar_type(2.1e9)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))


## Rest of problem

Just as before, we will continue to initialize the problem with all the same remaining parameters. Instead of separating the code into multiple cells, we will combine it all in one go and run the second half. You should see the result saved in a gif format called `Bridge_mesh_failure.gif`.

In [ ]:
# Define the weak form
u = fem.Function(V)  # Displacement
v = ufl.TestFunction(V) # Test function
u_t = fem.Function(V)  # Velocity
u_tt = fem.Function(V)  # Acceleration

I = ufl.variable(ufl.Identity(domain.geometry.dim)) 
F = ufl.variable(I + ufl.grad(u)) 
C = ufl.variable(F.T * F) 
Ic = ufl.variable(ufl.tr(C)) 
J = ufl.variable(ufl.det(F)) 
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
P = ufl.diff(psi, F) 

# Boundary conditions: Fix both ends of the bridge
def left_end(x):
    return np.isclose(x[0], 0.0)

def right_end(x):
    return np.isclose(x[0], L)


left_dofs = fem.locate_dofs_geometrical(V, left_end)
right_dofs = fem.locate_dofs_geometrical(V, right_end)
zero_displacement = np.array([0.0, 0.0, 0.0], dtype=default_scalar_type)
bcs = [fem.dirichletbc(zero_displacement, left_dofs, V),
       fem.dirichletbc(zero_displacement, right_dofs, V)]

# External force: Vertical force applied at the center of the bridge
force_center = fem.Constant(domain, default_scalar_type((0.0, 0.0, -1e4)))  # Force in the negative z-direction
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure("dx", domain=domain)
F_form = rho * ufl.dot(u_tt, v) * dx + ufl.inner(ufl.grad(v), P) * dx - ufl.dot(v, force_center) * ds

# Solver setup
problem = NonlinearProblem(F_form, u, bcs)
solver = NewtonSolver(domain.comm, problem)
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("Bridge_mesh_failure.gif", fps=10)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
function_grid["u"] = np.zeros((geometry.shape[0], 3))
function_grid.set_active_vectors("u")

actor = plotter.add_mesh(function_grid, show_edges=True, lighting=False, clim=[0, 0.1])

# Time-stepping parameters
dt = 1  # Time step size
T_end = 20.0  # Total simulation time
num_steps = int(T_end / dt)

# Time-stepping loop
log.set_log_level(log.LogLevel.INFO)
for step in range(num_steps):
    t = step * dt
    print(f"Time step {step + 1}/{num_steps}, Time: {t:.2f}s")
    
    # Solve for the displacement
    num_its, converged = solver.solve(u)
    assert converged, f"Solver did not converge at step {step + 1}"
    u.x.scatter_forward()

    # Update velocity and acceleration
    u_tt.x.array[:] = (u.x.array - u_t.x.array) / dt
    u_t.x.array[:] = u.x.array

    # Update visualization
    function_grid["u"][:, :3] = u.x.array.reshape(geometry.shape[0], 3)
    warped = function_grid.warp_by_vector("u", factor=1)
    plotter.update_coordinates(warped.points, render=False)
    plotter.write_frame()

plotter.close()

## Failure Condition 2 - Incorrect Tolerance

For our second example, we see fenicsx fail when we set the solver tolerance to be too sensitive (We can similarily set it to be very high and it will provide an inaccurate answer). It is virtually impossible for this framework to find solutions that have near 100 decimal points. It would simply take too long and error out. 

In [ ]:
# Bridge dimensions
L, W, H = 10.0, 1.0, 0.5  # Length, width, height
num_elements = [20, 4, 4]  # Coarser mesh resolution for P-mesh

# Create a 3D bridge mesh
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, W, H]], num_elements, mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))

# Material properties for bridge
rho = 7850  # Density
E = default_scalar_type(2.1e9)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))

# Define the weak form
u = fem.Function(V)  # Displacement
v = ufl.TestFunction(V) # Test function
u_t = fem.Function(V)  # Velocity
u_tt = fem.Function(V)  # Acceleration

I = ufl.variable(ufl.Identity(domain.geometry.dim)) 
F = ufl.variable(I + ufl.grad(u)) 
C = ufl.variable(F.T * F) 
Ic = ufl.variable(ufl.tr(C)) 
J = ufl.variable(ufl.det(F)) 
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
P = ufl.diff(psi, F) 

# Boundary conditions: Fix both ends of the bridge
def left_end(x):
    return np.isclose(x[0], 0.0)

def right_end(x):
    return np.isclose(x[0], L)


left_dofs = fem.locate_dofs_geometrical(V, left_end)
right_dofs = fem.locate_dofs_geometrical(V, right_end)
zero_displacement = np.array([0.0, 0.0, 0.0], dtype=default_scalar_type)
bcs = [fem.dirichletbc(zero_displacement, left_dofs, V),
       fem.dirichletbc(zero_displacement, right_dofs, V)]

# External force: Vertical force applied at the center of the bridge
force_center = fem.Constant(domain, default_scalar_type((0.0, 0.0, -1e4)))  # Force in the negative z-direction
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure("dx", domain=domain)
F_form = rho * ufl.dot(u_tt, v) * dx + ufl.inner(ufl.grad(v), P) * dx - ufl.dot(v, force_center) * ds

# Solver setup
problem = NonlinearProblem(F_form, u, bcs)
solver = NewtonSolver(domain.comm, problem)
solver.atol = 1e-100
solver.rtol = 1e-100
solver.convergence_criterion = "incremental"

pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("Bridge_tolerance_failure.gif", fps=10)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
function_grid["u"] = np.zeros((geometry.shape[0], 3))
function_grid.set_active_vectors("u")

actor = plotter.add_mesh(function_grid, show_edges=True, lighting=False, clim=[0, 0.1])

# Time-stepping parameters
dt = 1  # Time step size
T_end = 20.0  # Total simulation time
num_steps = int(T_end / dt)

# Time-stepping loop
log.set_log_level(log.LogLevel.INFO)
for step in range(num_steps):
    t = step * dt
    print(f"Time step {step + 1}/{num_steps}, Time: {t:.2f}s")
    
    # Solve for the displacement
    num_its, converged = solver.solve(u)
    assert converged, f"Solver did not converge at step {step + 1}"
    u.x.scatter_forward()

    # Update velocity and acceleration
    u_tt.x.array[:] = (u.x.array - u_t.x.array) / dt
    u_t.x.array[:] = u.x.array

    # Update visualization
    function_grid["u"][:, :3] = u.x.array.reshape(geometry.shape[0], 3)
    warped = function_grid.warp_by_vector("u", factor=1)
    plotter.update_coordinates(warped.points, render=False)
    plotter.write_frame()

plotter.close()

## Failure Condition 3 - Incorrect Problem Formulation

For our third and final example, if the problem statement is set up incorrectly, the remaining aadjustable parameters will most likely not work properly either. In the example below, we see an incorrect poisson ratio, bad youngs modulus, and an incredibly strong force applied. All of these conditions simply break fenicsx as the problem itself does not make sense. 

In [ ]:
# Bridge dimensions
L, W, H = 10.0, 1.0, 0.5  # Length, width, height
num_elements = [20, 4, 4]  # Coarser mesh resolution for P-mesh

# Create a 3D bridge mesh
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, W, H]], num_elements, mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))

# Material properties for bridge
rho = 200  # Density
E = default_scalar_type(2.1e3)
nu = default_scalar_type(0.9)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))

# Define the weak form
u = fem.Function(V)  # Displacement
v = ufl.TestFunction(V) # Test function
u_t = fem.Function(V)  # Velocity
u_tt = fem.Function(V)  # Acceleration

I = ufl.variable(ufl.Identity(domain.geometry.dim)) 
F = ufl.variable(I + ufl.grad(u)) 
C = ufl.variable(F.T * F) 
Ic = ufl.variable(ufl.tr(C)) 
J = ufl.variable(ufl.det(F)) 
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
P = ufl.diff(psi, F) 

# Boundary conditions: Fix both ends of the bridge
def left_end(x):
    return np.isclose(x[0], 0.0)

def right_end(x):
    return np.isclose(x[0], L)


left_dofs = fem.locate_dofs_geometrical(V, left_end)
right_dofs = fem.locate_dofs_geometrical(V, right_end)
zero_displacement = np.array([0.0, 0.0, 0.0], dtype=default_scalar_type)
bcs = [fem.dirichletbc(zero_displacement, left_dofs, V),
       fem.dirichletbc(zero_displacement, right_dofs, V)]

# External force: Vertical force applied at the center of the bridge
force_center = fem.Constant(domain, default_scalar_type((0.0, 0.0, -1e8)))  # Force in the negative z-direction
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure("dx", domain=domain)
F_form = rho * ufl.dot(u_tt, v) * dx + ufl.inner(ufl.grad(v), P) * dx - ufl.dot(v, force_center) * ds

# Solver setup
problem = NonlinearProblem(F_form, u, bcs)
solver = NewtonSolver(domain.comm, problem)
solver.atol = 1e-100
solver.rtol = 1e-100
solver.convergence_criterion = "incremental"

pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("Bridge_initialization_failure.gif", fps=10)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
function_grid["u"] = np.zeros((geometry.shape[0], 3))
function_grid.set_active_vectors("u")

actor = plotter.add_mesh(function_grid, show_edges=True, lighting=False, clim=[0, 0.1])

# Time-stepping parameters
dt = 1  # Time step size
T_end = 20.0  # Total simulation time
num_steps = int(T_end / dt)

# Time-stepping loop
log.set_log_level(log.LogLevel.INFO)
for step in range(num_steps):
    t = step * dt
    print(f"Time step {step + 1}/{num_steps}, Time: {t:.2f}s")
    
    # Solve for the displacement
    num_its, converged = solver.solve(u)
    assert converged, f"Solver did not converge at step {step + 1}"
    u.x.scatter_forward()

    # Update velocity and acceleration
    u_tt.x.array[:] = (u.x.array - u_t.x.array) / dt
    u_t.x.array[:] = u.x.array

    # Update visualization
    function_grid["u"][:, :3] = u.x.array.reshape(geometry.shape[0], 3)
    warped = function_grid.warp_by_vector("u", factor=1)
    plotter.update_coordinates(warped.points, render=False)
    plotter.write_frame()

plotter.close()